<a href="https://colab.research.google.com/github/lifepopkay/Tech-Monies/blob/Data-cleaning/Final_Data_cleanup_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final script used for cleaning the scrapped dataset from indeed

### Import packages

In [25]:
#import packages needed
import pandas as pd
import numpy as np
import re
import requests as rq

### Get the raw data from onedrive

In [26]:
# data url
url = "https://drive.google.com/u/0/uc?id=1Mx6ilfA0Ffm1s9ff1lgAiUnK8DxAPPBt&export=download"

# get response from the URL
response = rq.get(url)

# write content to excel file
with open("dataset.xlsx", "wb") as file:
    file.write(response.content)

# read data
data = pd.read_excel("dataset.xlsx")

data.head(5)

,title,id,company,location,link,salaryDesc,postDate,JobDesc,title scraped for,Country
0,Senior Software Engineer,job_3e1d31dda6c5cb4b,Offerzen,Lagos,/company/OfferZen/jobs/Senior-Software-Enginee...,Full-time,Just posted,The Opportunity\nOur newly formed Marketplace ...,Data Scientist,Nigeria
1,Data Scientist,job_c3e2ed6ba483528b,GVA Partners,Lagos,/company/GVA-Partners/jobs/Data-Scientist-c3e2...,Permanent +1,Today,Our client requires the skills of a Data Scien...,Data Scientist,Nigeria
2,Full stack developer,job_264deaa0926c095e,Horizonpay Nigeria Limited,Lagos,/company/HorizonPay-Nigeria-Limited/jobs/Full-...,"‚Ç¶700,000 a monthFull-time",Today,\nWork with development teams and product mana...,Data Scientist,Nigeria
3,Superintendent Pharmaceutical Officer,job_4ba7daa713f2751c,NaN,Lagos,/rc/clk?jk=4ba7daa713f2751c&fccid=dd616958bd9d...,Full-time,4 days ago,\n\n\n\nSuperintendent Pharmaceutical Officer\...,Data Scientist,Nigeria
4,Project HSE Advisor I,job_45b19f53adcf0053,Worley,Abeokuta,/rc/clk?jk=45b19f53adcf0053&fccid=d9805af20a6c...,Full-time,5 days ago,\n\n Company : Worley \n \n\nPrimary Location...,Data Scientist,Nigeria


### Salary and pay frequency Extraction
Extraction functions for pay frequency, and salary into upper & lower range columns

In [27]:
# functions 
def upper_salary_range(x):
    
    """
    This function extracts the upper salary range from the scrapped salary description and returns the upper salary.
    
    arg :
        x : the salary description scrapped (str)
    
    return:
        upper_salary_value : cleaned upper salary value (int)
    """
    
    scrapped_salary = str(x).split()
    upper_salary_value = ''
    lower_salary_value = ''
    trackposition = 0
    digit = ['0','1','2','3','4','5','6','7','8','9']
    
    for item in scrapped_salary:
        text_check = False
        #checking if the item in the list is possibliy salary( confirming if the 3 item is a number)
       
        
        try:
            text_check1= item[1]
            if text_check1 in digit and item[0] != '+':
                text_check = True
            else:
                try:
                    text_check2 = item[2]
                    if text_check2 in digit and item[0] != '+':
                        text_check = True
                    else:
                        try:
                            text_check3= item[3]
                            if text_check3 in digit and item[0] != '+':
                                text_check = True
                        except:
                            text_check = False
                except:
                    text_check = False
        except:
            text_check = False
            
            
        if trackposition == 0 and text_check == True:
            #lower_salary_value = t
             #remove leading letters from salary
            if item[0] in digit:
                lower_salary_value = item
            elif item[1] in digit:
                lower_salary_value = item[1:]
            elif item[2] in digit:
                lower_salary_value = item[2:]
            elif item[3] in digit:
                lower_salary_value = item[3:]
            
            # change tracker to shower lower_range is picked
            trackposition = 1    
            
            # find lower_salary_value range with upper_salary_value mergered togther
            for i in lower_salary_value:
                if i != ',' and i.lower() != 'k'and i != '.' and i not in digit:
                    ind = lower_salary_value.index(i)
                    upper_salary_value = lower_salary_value[ind:]
                    lower_salary_value = ''
                    
                    #clean currency from upper_salary_value
                    if upper_salary_value[0] in digit:
                        upper_salary_value = upper_salary_value
                    elif upper_salary_value[1] in digit:
                        upper_salary_value = upper_salary_value[1:]
                    elif upper_salary_value[2] in digit:
                        upper_salary_value = upper_salary_value[2:]
                    elif upper_salary_value[3] in digit:
                        upper_salary_value = upper_salary_value[3:]
                    elif upper_salary_value[4] in digit:
                        upper_salary_value = upper_salary_value[4:]
                    elif upper_salary_value[5] in digit:
                        upper_salary_value = upper_salary_value[5:]
                    
                    trackposition = 0  # toggle back tracker as upper has been picked
                    #exit the loop
                    break
                
            
        elif trackposition == 1 and text_check == True:
            if item[0] in digit and lower_salary_value != '':
                upper_salary_value = item
            elif item[1] in digit and lower_salary_value != '':
                upper_salary_value = item[1:]
            elif item[2] in digit and lower_salary_value != '':
                upper_salary_value = item[2:]
            elif item[3] in digit and lower_salary_value != '':
                upper_salary_value = item[3:]
                
                
     # replace ',' with '' 
    upper_salary_value = upper_salary_value.replace(',', '')
    
    #replace 'k' with '000'
    upper_salary_value = upper_salary_value.upper()
    if upper_salary_value[-1:] == 'K':
        upper_salary_value = upper_salary_value.replace('K', '')
        upper_salary_value = float(upper_salary_value)*1000
    
    #convert to int
    try:
        upper_salary_value = float(upper_salary_value)
    except:
        upper_salary_value = np.nan
    
    
    return upper_salary_value


def lower_salary_range(x):
        
    """
    This function extracts the lower salary range from the scrapped salary description and returns the lower salary.
    
    arg :
        x : the salary description scrapped (str)
    
    return:
        lower_salary_value : cleaned upper salary value (int)
    """
    scrapped_salary = str(x).split()   #split the salary description
    upper_salary_value = ''     #variable for holder upper salary range
    lower_salary_value = ''     #variable for holder lower salary range
    trackposition = 0        #variable for tracking when lower salary has been extracted 

    digit = ['0','1','2','3','4','5','6','7','8','9']  # number listing
    
    #looping through the description for the extraction 
    for item in scrapped_salary:
        text_check = False
        #checking if the item in the list is possibliy salary( confirming if the 3 item is a number)
       
        
        try:
            text_check1= item[1]
            if text_check1 in digit and item[0] != '+':
                text_check = True
            else:
                try:
                    text_check2 = item[2]
                    if text_check2 in digit and item[0] != '+':
                        text_check = True
                    else:
                        try:
                            text_check3= item[3]
                            if text_check3 in digit and item[0] != '+':
                                text_check = True
                        except:
                            text_check = False
                except:
                    text_check = False
        except:
            text_check = False
            
        
            
        
        #extraction based on tracker
        if trackposition == 0 and text_check == True:
            #remove leading letters from salary
            if item[0] in digit:
                lower_salary_value = item
            elif item[1] in digit:
                lower_salary_value = item[1:]
            elif item[2] in digit:
                lower_salary_value = item[2:]
            elif item[3] in digit:
                lower_salary_value = item[3:]
                
             # find lower range with upper mergered togther
            for i in lower_salary_value:
                if i != ',' and i.lower() != 'k' and i != '.'and i not in digit:
                    ind = lower_salary_value.index(i)
                    lower_salary_value = lower_salary_value[:ind]
                    upper_salary_value = lower_salary_value[ind:]
                    break
            
            #Update tracker 
            trackposition = 1
            
        elif trackposition == 1 and text_check == True:
            upper_salary_value = item
    
    # replace ',' with '' 
    lower_salary_value = lower_salary_value.replace(',', '')
    
    #and 'k' with '000'
    lower_salary_value = lower_salary_value.upper()
    if lower_salary_value[-1:] == 'K':
        lower_salary_value = lower_salary_value.replace('K', '')
        lower_salary_value = float(lower_salary_value)*1000
    
    #convert to int
    try:
        lower_salary_value = float(lower_salary_value)
    except:
        lower_salary_value = np.nan
        
    return lower_salary_value


def pay_freq(x):  
    """
    This function extracts the payment frequency from the scrapped salary description.
    
    arg :
        x : the salary description scrapped (str)
    
    return:
        results : payment frequency (str)
    """
    text = str(x).split()
    month_re = r'month\w*'
    year_re = r'year\w*'
    hour_re = r'hour\w*'

    results = None

    for t in text:
        month_finder = re.findall(month_re, t.lower())
        year_finder = re.findall(year_re, t.lower())
        hour_finder = re.findall(hour_re, t.lower())
        if len(year_finder) > 0:
            results = 'Yearly'
            break
        elif len(month_finder) > 0:
            results = 'Monthly'
            break
        elif len(hour_finder) > 0:
            results = 'Hourly'
    return results


#create and extract the upper and lower salary range for each row
data['upper_salary'] = data['salaryDesc'].apply(lambda x: upper_salary_range(x))
data['lower_salary'] = data['salaryDesc'].apply(lambda x: lower_salary_range(x))
data['payment_frequency'] = data['salaryDesc'].apply(lambda x: pay_freq(x))


data[['upper_salary', 'lower_salary','payment_frequency']].sample(20)

,upper_salary,lower_salary,payment_frequency
3131,NaN,NaN,None
991,NaN,NaN,None
2854,NaN,NaN,None
6374,174000.0,137000.0,Yearly
86,NaN,NaN,None
8638,135000.0,107000.0,Yearly
123,NaN,150000.0,Monthly
5155,93200.0,73600.0,Yearly
2538,NaN,NaN,None
588,NaN,NaN,None


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              8700 non-null   object 
 1   id                 8675 non-null   object 
 2   company            8574 non-null   object 
 3   location           8675 non-null   object 
 4   link               8675 non-null   object 
 5   salaryDesc         6589 non-null   object 
 6   postDate           8675 non-null   object 
 7   JobDesc            8675 non-null   object 
 8   title scraped for  8700 non-null   object 
 9   Country            8700 non-null   object 
 10  upper_salary       2397 non-null   float64
 11  lower_salary       2833 non-null   float64
 12  payment_frequency  2808 non-null   object 
dtypes: float64(2), object(11)
memory usage: 883.7+ KB


### Years of Experience Extraction
Extraction of years of experience 

In [29]:
def years_exp(x):
  global result
  expression = "\d*\+?\s+(?=year)"
  matches=re.search(expression,str(x))
  if matches:
    result = matches.group(0)
  else:
      None
  
  #remove the '+' on some of the years of experience
  try:
    if result.split()[-1][-1] == '+':
      result = result.split()[-1].replace('+', '')
  except:
    result = result

  
  #convert to integer
  try:
    result = int(result)
  except:
    result = np.nan

  return result
  

data['year_of_exp'] = data['JobDesc'].apply(lambda x: years_exp(x))

data[['year_of_exp']].sample(10)

,year_of_exp
3486,5.0
4877,2.0
6549,NaN
571,3.0
2964,NaN
1418,NaN
7651,2.0
6865,7.0
3328,NaN
1526,5.0


### Contract Type Extraction
Extraction of the Contract Type

In [30]:
def contract_type(x):  
    
    """
    This function extracts the contract type from the scrapped job salary description.
    
    arg :
        x : the job description scrapped (str)
    
    return:
        results : contract type (str)
    """
    
    #find 'Type:' index location
    try:
        pos = str(x).split().index('Type:')
        
        #get a list of all text after "Type:"
        text = str(x)[pos+1 : ].split()
    except:
        text = []
    
    
    
    full_time_re = r'full-time\w*'
    contract_re = r'contract\w*'

    results = None

    for t in text:
        full_time_finder = re.findall(full_time_re, t.lower())
        contract_finder = re.findall(contract_re, t.lower())
        #hour_finder = re.findall(hour_re, t.lower())
        if len(full_time_finder) > 0:
            results = 'Full-time'
        elif len(contract_finder) > 0:
            results = 'Contract'

    return results


#extract the contract type for each row
data['contract_type'] = data['JobDesc'].apply(lambda x: contract_type(x))

data[['contract_type']].sample(10)

,contract_type
8413,None
6760,None
463,None
8612,None
401,None
267,None
4737,None
7923,None
1775,Full-time
3416,None


### Clean title, Extract skills and eligibility
Clean title & extract skills & eligibility from jobDesc

In [31]:
def clean_data(df1):
  '''
  add new columns --> eligibilty, title_cleaned & skills_list
  takes dataframe as input & output dataframe
  '''

  # additional functions
  def replace_jd_keywords(text):
      for k,v in skillPat.items():
          text = re.sub(k, v, str(text),flags=re.I)
      return text
          
  def split_jd(text, key):
      tmp = []
      for i in text.split('!!!'):
          if i.startswith('['+key+']'):
              txt = re.sub('^\['+key+'\][ :,;]{0,5}','', i).strip().replace('\n',' ')
              tmp.append(txt) if len(txt)>0 else None
      
      if (key=='Requirements') & (len(tmp) == 0):
          tmp.append('[NO REQ]')
          for i in text.split('!!!'):
              if (not i.startswith('[')) & len(i.strip())>0:
                  tmp.append(i.strip().replace('\n',' '))
          
      return '| '.join(tmp)

  def search_skills(text):
    skillList = list()
    for skill in allSkills:
      if text.find(skill) != -1:
        skillList.append(skill)
    return list(set(skillList))

  # create patterns for eligibility
  eliPat = r'\n*(HS|HSD|High School Diploma|MBA|BS|MS|[Pp]h\.+[dD]\.+|[bB]achelor|[mM]aster|[uU]ndergraduate|B\.Sc\.).*[\.\n]{1}'
  replace_EP = {
      'undergraduate|bs|bachelor|b.sc.': 'undergraduate',
      'ph.d.|phd': 'doctorate',
      'mba|msc|ms|master': 'postgraduate',
      'HSD|High School Diploma': 'high school'
  }

  # titles
  titPat = {
          '[dD]ot\w?[nN]et': ".Net", # if contains Dot Net, Dotnet, etc. --> .Net
          '[Ss]r\.?': 'Senior', # Replace with Senior
          '[Jj]r\.?': 'Junior', # Replace with Junior
          'Financial Advisory International |USARPC ': '', # remove company name
        # Garbage value cleanups
          '\([\w\d ,\-]*\)|URGENT|\!|\-\-|\-$|\u200b|\– part':'',
          '\|.*':'', # remove everything after "|"
          ' for .*| in .*| at .*':'', # remove everything after "for" # remove everything after "for", "in", "at"
          '[Rr]emote': '', # remove remote
          ' , .*':'', # remove everything after comma
          '\[.*\]':'', # remove everything within brackets
          '\s[Jj]ob.*$':'', # remove everything after job word
          '_|\-|#.*|\(.*\)?':' ', #remove underscores & dash
          ' Pune.*$| Mumbai.*$| Hyd.*$| New Delhi.*$| Bangalore.*$': '', # remove places
          ' Marketing Measure.*$|¬£.*$| ‚Ä.*$|, New.*$| year.*$| \d+.*$| :.*$| \w+\d+.*$': '', # remove from end
          '^.*Hub |^.*\: ': '', # remove from start
        }
  
  # skill block
  skillPat = {
        'the role[ ]?[\n\:·]|your role[ ]?[\n\:·]|Job Scope|trust you to|What You[\'Äô]ll Do|Job Highlight|Summary|Responsibilities[\n\:·]|Responsible for|What to expect|Job Description|you will work': "!!![Role]",
        'need to have|What You Have|you must have|You have[ ]?[\n\:·]|Your expertise|Requirement[s]?[ ]?[\n\:·]|Requirements(?! )|Job Requirement|Desired Competencies|What we seek|Who You Are|What you\'ll need|About you|Qualifications': "!!![Requirements]", 
        'What You\'ll Enjoy|Benefits|What we offer|we will offer': "!!![Benefits]", 
        'Apply if|Apply Now|Please send|Please contact|Please Provide|Join Us|Closing Statement|Application Method|is one of the leading|about us':"!!![End]"
      }

  # skills
  allSkills = ['.NET','.net','3d','3rd party Ad serving platforms','A/B Testing','A/B testing','ABAP','ADO.NET','AI Programming',
             'ALGOL','APL','ASCII Encoding','ASP / ASP.NET','ATL','AWS','Action Oriented','ActionScript','Ad Campaign Management',
             'Ad Placement','Ada','Adhoc Analysis','Adhoc copy-writing','Advanced Excel','Adwords','Algebra','Algorithms','Alice',
             'Analytical Skills','Articulation','Assembly Language','Assembly and product QA activities','Awk','B2B Sales','BA',
             'BBC Basic','BTL','Backbone.js','Bagging','Balancing Short term and long term solutions','Balsamiq','Bias for action',
             'Books','Brand Collaboration','Brand Management','Build Customer Insights','Build MVP or POC','Business Mindset',
             'Business Plan','Business insights','C','C#','C++','CGI','COBOL','CORBA','CSS','CVS','CakePHP','Calculus','Campaign Management',
             'Career development of juniors','Cascading Style Sheets','Chat-bots','Clevertap','Client Management','Clustering','Cocoa',
             'CodeIgniter','Collaboration with Designers','Collaboration with data scientists','Collaboration with designers',
             'Collaboration with developer','Collateral Branding','Communication Skills','Communication Strategy','Community Building',
             'Community Development','Competitor research','Computer Knowledge','Computer Vision','Consultative Sales','Consumer research',
             'Consumer segmentation','Container technologies','Content Marketing','Content Operations','Cookies','Coordination Skills',
             'Creative','Crimson Hexagon','Critical Thinking','Cross-functional ','Customer Focused','Customer Relationship management',
             'Customer Training','D','D3.js','DOM','Dash','Dashboards','Data Analysis','Data Architecture','Data Assets','Data Automation',
             'Data Driven','Data Modeling','Data Reporting','Data Visualization','Data Warehousing','Data driven','Data engineering',
             'Data mining','Databases','Deal with ambiguity','Decision Trees','Deep Learning','Delphi','Demand Forecasting',
             'Demand Planning','Detail-Oriented','DevOps','Digital marketing','DigitalOcean','Dimensionality reduction','Direct Sales',
             'Distributed Computing','Django','Docker','Documentation','Dreamweaver','ERP System','ETL','Email Marketing','Ensemble Modeling',
             'Entity Recognition','Entrepreneurial mind-set','Entrepreneurship mentality','Erlang and Elixir','Event collaboration','Excel',
             'Execute marketing campaigns','Experience with Technology/Software','Experienced with High Volume/Production Environment',
             'Express.js','Extreme Programming','F#','FFmpeg','FORTH','FORTRAN','Facebook Insights','Familiar with Hubspot','Fan building',
             'FastAPI','First principle thinking','Flask','Flexible','Fraud Detection','Functional Programming','GATE','GNUstep',
             'General_Awareness','Geospatial Data','Git','Go','Google Ad-word Campaign','Google Analytics','Gradient Boosting algorithm',
             'Gradient Descent','Growth Strategy','Gurobi','HDFS','HTML','Hadoop','Handling Large Amounts of Data','Hands on work experience',
             'Haskell','Header Bidding','High Budget Campaigns','High ROI on campaigns','Hive','Hypothesis testing','IDL','INTERCAL','IOT',
             'ITIL','Image Analytics','ImageMagick','InVision','Increase Brand Awareness','Indentify growth potential','Industry knowledge',
             'Influencer Marketing','Initiative','Institutional Sales','Integrity and Trust','Interpersonal Skills','Inventory Management',
             'JSON','Java','Javascript','KNN','Keras','Knowledge of Agile process and principles','Knowledge of Technology ',
             'Knowledge of Web Applications','Knowledge of ticketing tools','LabVIEW','Laravel','Lavarel','Lead Generation',
             'Leadership','Lean manufacturing processes','Lidar','Linear algebra','Linked Lists','Lisp','List of Programming Tools and Libraries',
             'Logistics Management','Logo','Loyalty program','MDN','ML','MPI','MS Access','MS Excel','MS Office','MS Word','MSXML','Machine Learning',
             'Manage platform operations','Manage warehouse activities','Managing Marketing Databases','Managing Product Backlogs',
             'Managing and measuring work ','MantisBT','Market Research','Marketing','Marketing Automation','Marketing Mix','Media Mix',
             'Mentoring/Coaching skills','Mercurial','MetaQuotes Language','Metabase','Minimize Loss','Model Evaluation','Modula-3',
             'MongoDB','Monitoring Skills','Multi Tasking','MxNet','MySQL','NLP','NLTK','NLU','NXT-G','Naive Bayes','Natural Language Processing',
             'Ncurses','Negotiation Skills','Neo4j','NetCDF','Network Programming','Neural Networks','New Product Launch','NoSQL','Node.js','OAuth',
             'OCR','OCaml','OS Development','Object detection','Object-Oriented Programming','Objective-C',
             'One extra international language apart from English','Open CV','Open Ended problem Solution','OpenCL','OpenCV','OpenID','OpenSSL',
             'Optimization','Organizational','Organized','Organized/ Detail Oriented','Ownership','PDM System','PHP','PHProjekt','PL/I','PL/SQL',
             'PLM System','PMP Certification','PRD Development','PROLOG','Partner Relationship management','Pascal','People Management','People focused',
             'People orientation','Perl','Perseverance','Plotly','PostScript','PostgreSQL','PowerBI','PowerPoint','Predictive modeling',
             'Prescriptive Analytics','Presentation','Price Modeling','Probability','Problem Solving and Decision Making','Problem structuring',
             'Process Management','Process Orientation','Process automation','Product Design','Product Metrics','Product Ownership',
             'Product Strategic Direction','Product road-map','Project Management','Promotional Budget planning','Property sourcing',
             'Protocols','Pure Data','PySpark','PyTorch','Python','Quantitative Analytics','Quick/good Learner','R','Random Forest','RapidWeaver',
             'RavenDB','Recommender System','Recruitment and on boarding','Redis','RegEx','Regression','Reinforcement Learning',
             'Reporting/Forecasting','Resource Planning','Result-oriented','Retention marketing','Revenue Management','Rexx',
             'Roaster Management','Robots','Ruby on Rails','Ruby on rails','S-PLUS','SAS','SDLC','SEO','SGML','SLAM','SMIL',
             'SNOBOL','SOAP','SOP creation and implementation','SPSS','SQL','SQLite','SSH','SSI','SVM','SaaS Product',
             'Sampling Techniques','Scala','Seaborn','Sed','Semantic Analysis','Semi-Supervised Learning','Should be able to work independently',
             'Simula','Simultaneous localization and mapping','Site Catalyst','Six Sigma Certification','Smalltalk','Social Media Platforms',
             'Social Media/ Web Services','Social Network Analysis','Software development','Sorting Algorithms','Spark','Speech Recognition',
             'Spotfire','Sqoop','Stakeholder Management','Stata','Statistics','Strategic Thinking','Strategies','Structured data','Subversion',
             'Supervised Learning','Supervising Skills','Supply Chain','Swift','Tableau','Task Oriented','Task prioritization','Tcl/Tk',
             'TeX and LaTeX','Team Management','Team Work','Team building','Team leader','Technical concept understanding',
             'Techniques to improve stock availability','TensorFlow','Text Image processing','Text Mining','Theano',
             'Third Party Integration','Time management skill','Time series analysis','Timely task completion','Tracking','UI','URL','UX',
             'Understanding Stakeholders','Understanding of "Web 2.0"','Understanding of claims operations','Unified Modeling Language',
             'Unix Shells','Unstructured data','Unsupervised Learning','Updated with latest industry techniques','User Profiling',
             'User Research','VBA','VHDL','VRML','Verilog','Vi','Visual Basic','Visual FoxPro','WAP/WML','WCF','WSDL','WSGI','Web Analytics',
             'Web Product','Web Standards','Web apps','WebKit Web Inspector','Website optimization','Weka','Wire-frame',
             'Work under pressure in short timelines','Writing Skills','XGBoost','XML','XSL','YUI','Zero to one problems',
             'Zikula','alliances with media partners','angular','api','asp','bundle adjustment','can-do attitude','client',
             'copy-editing','creative thinking','customer focused','ddp','end to end ownership','firebase','fleet management','flutter',
             'go-getter attitude','high tolerance to ambiguity','http','jQuery','js','json','matplotlib','meteor','mock up design','mongo',
             'node','object oriented design','outlier detection','performance metrics','problem solver','process oriented','product testing',
             'qualitative & quantitative research','scikit-image','sentiment analysis','server','sfm','social media best practices',
             'structure from motion','trade offs','user understanding','wordpress']

  # apply patterns & extract columns
  # eligibility
  df1['eligibility'] = df1['JobDesc'].str.extract(eliPat)
  df1['eligibility'] = df1['eligibility'].str.lower().replace(replace_EP, regex=True)
  print("Total {} rows with {} unique values of eligibilities.".format(len(df1['eligibility']),len(df1['eligibility'].unique())))

  # titles
  df1['title_cleaned'] = df1['title'].replace(titPat, regex=True).str.strip().replace({'^,|,$|\–$|\-$|,$':''},regex=True).str.strip()
  df1[['title','title_cleaned']].tail(50)
  print("Total {} rows with {} unique values of cleaned titles".format(len(df1['title_cleaned']),len(df1['title_cleaned'].unique())))

  # skills
  df1['JD_cleaned'] = df1['JobDesc'].apply(replace_jd_keywords)
  df1['skills'] = df1['JD_cleaned'].apply(lambda x: split_jd(x,'Requirements')).str.strip()
  df1['skills_list'] = df1['skills'].apply(search_skills)

  return df1.drop(['JD_cleaned', 'skills'], axis=1)


#Pass in the dataset to the function 
clean_data(data)

data.head(5)

Total 8700 rows with 6 unique values of eligibilities.
Total 8700 rows with 4381 unique values of cleaned titles


,title,id,company,location,link,salaryDesc,postDate,JobDesc,title scraped for,Country,upper_salary,lower_salary,payment_frequency,year_of_exp,contract_type,eligibility,title_cleaned,JD_cleaned,skills,skills_list
0,Senior Software Engineer,job_3e1d31dda6c5cb4b,Offerzen,Lagos,/company/OfferZen/jobs/Senior-Software-Enginee...,Full-time,Just posted,The Opportunity\nOur newly formed Marketplace ...,Data Scientist,Nigeria,NaN,NaN,None,5.0,Full-time,NaN,Senior Software Engineer,The Opportunity\nOur newly formed Marketplace ...,". Where needed, you are producing easy to unde...","[D, AWS, C, asp, R]"
1,Data Scientist,job_c3e2ed6ba483528b,GVA Partners,Lagos,/company/GVA-Partners/jobs/Data-Scientist-c3e2...,Permanent +1,Today,Our client requires the skills of a Data Scien...,Data Scientist,Nigeria,NaN,NaN,None,4.0,None,undergraduate,Data Scientist,Our client requires the skills of a Data Scien...,"¬∑ B.Sc. in Statistics, Mathematics, Engineeri...","[PostgreSQL, PL/SQL, Prescriptive Analytics, E..."
2,Full stack developer,job_264deaa0926c095e,Horizonpay Nigeria Limited,Lagos,/company/HorizonPay-Nigeria-Limited/jobs/Full-...,"‚Ç¶700,000 a monthFull-time",Today,\nWork with development teams and product mana...,Data Scientist,Nigeria,NaN,700000.0,Monthly,3.0,Full-time,undergraduate,Full stack developer,\nWork with development teams and product mana...,[NO REQ],[R]
3,Superintendent Pharmaceutical Officer,job_4ba7daa713f2751c,NaN,Lagos,/rc/clk?jk=4ba7daa713f2751c&fccid=dd616958bd9d...,Full-time,4 days ago,\n\n\n\nSuperintendent Pharmaceutical Officer\...,Data Scientist,Nigeria,NaN,NaN,None,2.0,None,undergraduate,Superintendent Pharmaceutical Officer,\n\n\n\nSuperintendent Pharmaceutical Officer\...,"Bachelor‚Äôs degree in pharmacy, At least 2 y...","[Presentation, Time management skill, C, Excel..."
4,Project HSE Advisor I,job_45b19f53adcf0053,Worley,Abeokuta,/rc/clk?jk=45b19f53adcf0053&fccid=d9805af20a6c...,Full-time,5 days ago,\n\n Company : Worley \n \n\nPrimary Location...,Data Scientist,Nigeria,NaN,NaN,None,5.0,None,hs,Project HSE Advisor I,\n\n Company : Worley \n \n\nPrimary Location...,[NO REQ]| Company : Worley Primary Locatio...,"[MS Office, D, C, asp, R]"


### Drop old columns not needed from data set

In [32]:
cleaned_df = data.copy()

In [33]:
cleaned_df.columns

Index(['title', 'id', 'company', 'location', 'link', 'salaryDesc', 'postDate',
       'JobDesc', 'title scraped for', 'Country', 'upper_salary',
       'lower_salary', 'payment_frequency', 'year_of_exp', 'contract_type',
       'eligibility', 'title_cleaned', 'JD_cleaned', 'skills', 'skills_list'],
      dtype='object')

In [34]:
cleaned_df = cleaned_df.drop(['title','id','link','salaryDesc','postDate','JobDesc','JD_cleaned', 'skills'], axis = 1)

In [35]:
cleaned_df.head()

,company,location,title scraped for,Country,upper_salary,lower_salary,payment_frequency,year_of_exp,contract_type,eligibility,title_cleaned,skills_list
0,Offerzen,Lagos,Data Scientist,Nigeria,NaN,NaN,None,5.0,Full-time,NaN,Senior Software Engineer,"[D, AWS, C, asp, R]"
1,GVA Partners,Lagos,Data Scientist,Nigeria,NaN,NaN,None,4.0,None,undergraduate,Data Scientist,"[PostgreSQL, PL/SQL, Prescriptive Analytics, E..."
2,Horizonpay Nigeria Limited,Lagos,Data Scientist,Nigeria,NaN,700000.0,Monthly,3.0,Full-time,undergraduate,Full stack developer,[R]
3,NaN,Lagos,Data Scientist,Nigeria,NaN,NaN,None,2.0,None,undergraduate,Superintendent Pharmaceutical Officer,"[Presentation, Time management skill, C, Excel..."
4,Worley,Abeokuta,Data Scientist,Nigeria,NaN,NaN,None,5.0,None,hs,Project HSE Advisor I,"[MS Office, D, C, asp, R]"


In [36]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   company            8574 non-null   object 
 1   location           8675 non-null   object 
 2   title scraped for  8700 non-null   object 
 3   Country            8700 non-null   object 
 4   upper_salary       2397 non-null   float64
 5   lower_salary       2833 non-null   float64
 6   payment_frequency  2808 non-null   object 
 7   year_of_exp        6758 non-null   float64
 8   contract_type      1267 non-null   object 
 9   eligibility        5350 non-null   object 
 10  title_cleaned      8700 non-null   object 
 11  skills_list        8700 non-null   object 
dtypes: float64(3), object(9)
memory usage: 815.8+ KB
